# Análisis de Flujos de Dinero en Ekhilur

Este notebook realiza un análisis detallado de las transacciones financieras en la plataforma Ekhilur, utilizando consultas directas a la base de datos MySQL.

## 1. Configuración Inicial

Primero importamos las librerías necesarias y establecemos la conexión con la base de datos.

In [1]:
# Importar librerías necesarias
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuración de visualización
plt.style.use('seaborn')
sns.set_palette('husl')
%matplotlib inline

# Configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1802020149.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


### Función para conectar a la base de datos

Creamos una función helper para manejar las consultas SQL.

In [2]:
import os
from dotenv import load_dotenv
import mysql.connector
import pandas as pd

# Cargar variables de entorno
load_dotenv()

def get_db_connection():
    return mysql.connector.connect(
        host='localhost',
        port=3308,
        user='user',
        password='userpassword',
        database='ekhilur'
    )

def ejecutar_query(query):
    """Ejecuta una query SQL y devuelve los resultados como DataFrame"""
    conn = get_db_connection()
    try:
        return pd.read_sql_query(query, conn)
    finally:
        conn.close()

In [3]:
# Consulta para verificar consistencia de datos
query_consistencia = """
WITH stats AS (
    SELECT 
        COUNT(*) as total_registros,
        COUNT(DISTINCT Id_fecha) as dias_unicos,
        COUNT(DISTINCT Id_tipo_operacion) as tipos_operacion_unicos,
        COUNT(DISTINCT Usuario_emisor) as emisores_unicos,
        COUNT(DISTINCT Usuario_receptor) as receptores_unicos,
        SUM(Cantidad) as suma_total,
        AVG(Cantidad) as promedio_operacion,
        MIN(Cantidad) as min_cantidad,
        MAX(Cantidad) as max_cantidad,
        COUNT(CASE WHEN Cantidad < 0 THEN 1 END) as operaciones_negativas,
        COUNT(CASE WHEN Cantidad > 0 THEN 1 END) as operaciones_positivas,
        COUNT(CASE WHEN Cantidad = 0 THEN 1 END) as operaciones_cero
    FROM fact_table
),
balance_por_tipo AS (
    SELECT 
        o.Operacion,
        COUNT(*) as num_operaciones,
        SUM(f.Cantidad) as total_por_tipo,
        AVG(f.Cantidad) as promedio_por_tipo,
        MIN(f.Cantidad) as min_por_tipo,
        MAX(f.Cantidad) as max_por_tipo
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    GROUP BY o.Operacion
),
balance_por_fecha AS (
    SELECT 
        SUBSTRING(Id_fecha, 1, 4) as año,
        COUNT(*) as operaciones_por_año,
        SUM(Cantidad) as total_por_año
    FROM fact_table
    GROUP BY SUBSTRING(Id_fecha, 1, 4)
)
SELECT 
    'Estadísticas Generales' as categoria,
    total_registros,
    dias_unicos,
    tipos_operacion_unicos,
    emisores_unicos,
    receptores_unicos,
    suma_total,
    promedio_operacion,
    min_cantidad,
    max_cantidad,
    operaciones_negativas,
    operaciones_positivas,
    operaciones_cero
FROM stats;
"""

# Ejecutar la consulta
df_consistencia = ejecutar_query(query_consistencia)

# Mostrar resultados
print("\n=== ANÁLISIS DE CONSISTENCIA DE DATOS ===")
print("\nEstadísticas Generales:")
print("-" * 50)
for columna in df_consistencia.columns:
    if columna != 'categoria':
        valor = df_consistencia[columna].iloc[0]
        if isinstance(valor, (int, float)):
            if columna in ['suma_total', 'promedio_operacion', 'min_cantidad', 'max_cantidad']:
                print(f"{columna}: {valor:,.2f} €")
            else:
                print(f"{columna}: {valor:,}")
        else:
            print(f"{columna}: {valor}")

# Consulta adicional para balance por tipo de operación
query_balance_tipo = """
SELECT 
    o.Operacion,
    COUNT(*) as num_operaciones,
    SUM(f.Cantidad) as total_por_tipo,
    AVG(f.Cantidad) as promedio_por_tipo,
    MIN(f.Cantidad) as min_por_tipo,
    MAX(f.Cantidad) as max_por_tipo
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY total_por_tipo DESC;
"""

df_balance_tipo = ejecutar_query(query_balance_tipo)

print("\nBalance por Tipo de Operación:")
print("-" * 50)
print(df_balance_tipo.to_string(index=False))

# Consulta para balance por año
query_balance_anual = """
SELECT 
    SUBSTRING(Id_fecha, 1, 4) as año,
    COUNT(*) as operaciones_por_año,
    SUM(Cantidad) as total_por_año
FROM fact_table
GROUP BY SUBSTRING(Id_fecha, 1, 4)
ORDER BY año;
"""

df_balance_anual = ejecutar_query(query_balance_anual)

print("\nBalance por Año:")
print("-" * 50)
print(df_balance_anual.to_string(index=False))

# Visualización del balance por tipo de operación
fig = go.Figure(data=[
    go.Bar(
        name='Total por Tipo',
        x=df_balance_tipo['Operacion'],
        y=df_balance_tipo['total_por_tipo'],
        text=df_balance_tipo['total_por_tipo'].round(2),
        textposition='auto',
    )
])

fig.update_layout(
    title='Balance Total por Tipo de Operación',
    xaxis_title='Tipo de Operación',
    yaxis_title='Total (€)',
    template='plotly_white'
)

fig.show()

# Visualización del balance anual
fig2 = go.Figure(data=[
    go.Bar(
        name='Total por Año',
        x=df_balance_anual['año'],
        y=df_balance_anual['total_por_año'],
        text=df_balance_anual['total_por_año'].round(2),
        textposition='auto',
    )
])

fig2.update_layout(
    title='Balance Total por Año',
    xaxis_title='Año',
    yaxis_title='Total (€)',
    template='plotly_white'
)

fig2.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)



=== ANÁLISIS DE CONSISTENCIA DE DATOS ===

Estadísticas Generales:
--------------------------------------------------
total_registros: 340322
dias_unicos: 366
tipos_operacion_unicos: 12
emisores_unicos: 1530
receptores_unicos: 1339
suma_total: 6,500,191.99 €
promedio_operacion: 19.10 €
min_cantidad: 0.00 €
max_cantidad: 15,000.00 €
operaciones_negativas: 0
operaciones_positivas: 340244
operaciones_cero: 78

Balance por Tipo de Operación:
--------------------------------------------------
              Operacion  num_operaciones  total_por_tipo  promedio_por_tipo  min_por_tipo  max_por_tipo
         Pago a usuario           112779      3430730.90          30.419944          0.00       2060.00
         Conversión a €              969      1267647.50        1308.201754          1.00      15000.00
    Recarga por tarjeta             9306      1255837.00         134.949173         50.00        500.00
         Cobro desde QR            12894       299646.69          23.239235          0.01 

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)



Balance por Año:
--------------------------------------------------
 año  operaciones_por_año  total_por_año
2024               340322     6500191.99


## 2. Análisis de Tipos de Operaciones

Analizamos la distribución de las operaciones y sus importes asociados.

In [4]:
# Consulta para obtener los tipos de operaciones y sus estadísticas
query_operaciones = """
SELECT 
    o.Operacion,
    COUNT(*) as total_operaciones,
    SUM(f.Cantidad) as suma_total,
    AVG(f.Cantidad) as promedio_importe
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY o.Operacion
ORDER BY suma_total DESC;
"""

df_operaciones = ejecutar_query(query_operaciones)

# Crear gráfico de barras con Plotly
fig = go.Figure(data=[
    go.Bar(name='Suma Total', x=df_operaciones['Operacion'], y=df_operaciones['suma_total'])
])

fig.update_layout(
    title='Volumen Total por Tipo de Operación',
    xaxis_title='Tipo de Operación',
    yaxis_title='Suma Total (€)',
    template='plotly_white'
)

fig.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



## 3. Análisis Temporal de Transacciones

Analizamos cómo evolucionan las transacciones a lo largo del tiempo.

In [5]:
# Consulta para análisis temporal
query_temporal = """
SELECT 
    DATE(CONCAT(
        SUBSTRING(f.Id_fecha, 1, 4), '-',
        SUBSTRING(f.Id_fecha, 5, 2), '-',
        SUBSTRING(f.Id_fecha, 7, 2)
    )) as fecha,
    o.Operacion,
    SUM(f.Cantidad) as suma_diaria
FROM fact_table f
JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
GROUP BY fecha, o.Operacion
ORDER BY fecha;
"""

df_temporal = ejecutar_query(query_temporal)

# Crear gráfico de líneas temporal
fig = px.line(df_temporal, 
              x='fecha', 
              y='suma_diaria', 
              color='Operacion',
              title='Evolución Temporal de Transacciones por Tipo')

fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Importe Total Diario (€)',
    template='plotly_white'
)

fig.show()

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



## 4. Análisis de Entradas y Salidas de Dinero

En el circuito Ekhilur existen 13 tipos diferentes de operaciones que afectan de manera distinta al flujo de dinero. Es importante entender cómo cada operación impacta en la masa monetaria del sistema:

### 4.1 Operaciones que AUMENTAN la masa monetaria
1. **Recarga por tarjeta (ID: 6)**
   - Entrada directa de dinero al circuito
   - Usuario origen es siempre el mismo (sistema)
   - Usuario destino puede ser particular o profesional

2. **Bonificación por compra (ID: 4)**
   - Entrada de dinero desde el ayuntamiento
   - Usuario origen es siempre la cooperativa ekhilur
   - Límites:
     * 100€ por usuario/mes
     * 12,500€ mensuales del ayuntamiento total

### 4.2 Operaciones que REDUCEN la masa monetaria
1. **Conversión a € (ID: 2)**
   - Indicador principal de salidas de dinero
   - Solo para cuentas profesionales
   - Usuario origen y destino son el mismo
   - Marca el dinero para su posterior retirada

2. **Comisión por retirada (ID: 3)**
   - 1% + IVA del total retirado
   - Usuario destino siempre es la cooperativa

3. **Cuota mensual de socio (ID: 5)**
   - Cuota de mantenimiento
   - Usuario destino es la cooperativa

4. **Cuota variable (ID: 10)**
   - 0.5% del saldo medio mensual para comercios
   - Usuario destino es la cooperativa

5. **Retirada a cuenta bancaria (ID: 12)**
   - Salida efectiva del dinero del sistema
   - Relacionada con la Conversión a €

### 4.3 Operaciones NEUTRAS (no afectan a la masa monetaria)
1. **Pago a usuario (ID: 1)**
   - Puede ser:
     * Pago normal (particular→profesional o entre profesionales)
     * Recarga (profesional→particular)
     * Bizum (particular→particular)

2. **Descuento automático (ID: 0)**
   - Bonificación del comercio al usuario
   - Efecto interno

3. **Cobro desde QR (ID: 7)**
   - Similar al pago a usuario
   - Método alternativo de cobro

4. **Cuota de socio (ID: 8)**
   - Pago único de 10€
   - Movimiento interno

5. **Transferencia interna (ID: 9)**
   - Movimiento neutro que puede ignorarse

6. **Donación (ID: 11)**
   - 0.5% del saldo medio mensual
   - Movimiento interno entre usuarios

En las siguientes secciones analizaremos en detalle estos flujos de dinero, centrándonos especialmente en:
1. El balance neto mensual (entradas - salidas)
2. La evolución de las bonificaciones y su cumplimiento de límites
3. El patrón de retiradas de dinero del sistema

### 4.4 Balance Mensual de Masa Monetaria

Para entender el flujo de dinero en el circuito, analizaremos:

1. **Entradas mensuales**:
   - Recargas por tarjeta
   - Bonificaciones por compra

2. **Salidas mensuales**:
   - Conversiones a euros
   - Comisiones por retirada
   - Cuotas mensuales de socio
   - Cuotas variables

3. **Indicadores clave**:
   - Balance neto mensual
   - Tendencia de entradas y salidas
   - Proporción entre tipos de operaciones

In [6]:
# Consulta SQL para obtener los datos
query_balance_mensual = """
WITH datos_mensuales AS (
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        CASE SUBSTRING(f.Id_fecha, 5, 2)
            WHEN '01' THEN 'Enero'
            WHEN '02' THEN 'Febrero'
            WHEN '03' THEN 'Marzo'
            WHEN '04' THEN 'Abril'
            WHEN '05' THEN 'Mayo'
            WHEN '06' THEN 'Junio'
            WHEN '07' THEN 'Julio'
            WHEN '08' THEN 'Agosto'
            WHEN '09' THEN 'Septiembre'
            WHEN '10' THEN 'Octubre'
            WHEN '11' THEN 'Noviembre'
            WHEN '12' THEN 'Diciembre'
        END as nombre_mes,
        -- Entradas de dinero
        SUM(CASE 
            WHEN o.Operacion = 'Recarga por tarjeta' THEN f.Cantidad
            ELSE 0 
        END) as recarga_tarjeta,
        
        SUM(CASE 
            WHEN o.Operacion = 'Bonificación por compra' THEN f.Cantidad
            ELSE 0 
        END) as bonificaciones,
        
        -- Salidas de dinero
        SUM(CASE 
            WHEN o.Operacion = 'Conversión a €' THEN f.Cantidad
            ELSE 0 
        END) as conversion_euros,
        
        SUM(CASE 
            WHEN o.Operacion = 'Comisión por retirada' THEN f.Cantidad
            ELSE 0 
        END) as comisiones,
        
        SUM(CASE 
            WHEN o.Operacion = 'Cuota mensual de socio' THEN f.Cantidad
            ELSE 0 
        END) as cuotas_mensuales,
        
        SUM(CASE 
            WHEN o.Operacion = 'Cuota variable' THEN f.Cantidad
            ELSE 0 
        END) as cuotas_variables
        
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6), 
             CASE SUBSTRING(f.Id_fecha, 5, 2)
                WHEN '01' THEN 'Enero'
                WHEN '02' THEN 'Febrero'
                WHEN '03' THEN 'Marzo'
                WHEN '04' THEN 'Abril'
                WHEN '05' THEN 'Mayo'
                WHEN '06' THEN 'Junio'
                WHEN '07' THEN 'Julio'
                WHEN '08' THEN 'Agosto'
                WHEN '09' THEN 'Septiembre'
                WHEN '10' THEN 'Octubre'
                WHEN '11' THEN 'Noviembre'
                WHEN '12' THEN 'Diciembre'
             END
)
SELECT 
    nombre_mes,
    -- Entradas
    ROUND(recarga_tarjeta, 2) as recarga_tarjeta,
    ROUND(bonificaciones, 2) as bonificaciones,
    ROUND(recarga_tarjeta + bonificaciones, 2) as total_entradas,
    
    -- Salidas
    ROUND(conversion_euros, 2) as conversion_euros,
    ROUND(comisiones, 2) as comisiones,
    ROUND(cuotas_mensuales, 2) as cuotas_mensuales,
    ROUND(cuotas_variables, 2) as cuotas_variables,
    ROUND(conversion_euros + comisiones + cuotas_mensuales + cuotas_variables, 2) as total_salidas,
    
    -- Balance
    ROUND((recarga_tarjeta + bonificaciones) - 
          (conversion_euros + comisiones + cuotas_mensuales + cuotas_variables), 2) as balance_neto
FROM datos_mensuales
ORDER BY año_mes;
"""

# Ejecutar la consulta
df_balance = ejecutar_query(query_balance_mensual)

# Crear visualización con Plotly
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Balance Mensual de Masa Monetaria', 'Desglose de Entradas y Salidas'),
    vertical_spacing=0.15,
    specs=[[{"type": "scatter"}],
           [{"type": "bar"}]]
)

# Gráfico de líneas para el balance general
fig.add_trace(
    go.Scatter(
        x=df_balance['nombre_mes'],
        y=df_balance['total_entradas'],
        name='Entradas',
        line=dict(color='green', width=2),
        fill='tozeroy'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_balance['nombre_mes'],
        y=df_balance['total_salidas'],
        name='Salidas',
        line=dict(color='red', width=2),
        fill='tozeroy'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_balance['nombre_mes'],
        y=df_balance['balance_neto'],
        name='Balance Neto',
        line=dict(color='blue', width=2, dash='dash')
    ),
    row=1, col=1
)

# Gráfico de barras para el desglose
fig.add_trace(
    go.Bar(
        x=df_balance['nombre_mes'],
        y=df_balance['recarga_tarjeta'],
        name='Recargas',
        marker_color='lightgreen'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        x=df_balance['nombre_mes'],
        y=df_balance['bonificaciones'],
        name='Bonificaciones',
        marker_color='darkgreen'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        x=df_balance['nombre_mes'],
        y=-df_balance['conversion_euros'],
        name='Conversiones €',
        marker_color='salmon'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Bar(
        x=df_balance['nombre_mes'],
        y=-df_balance['comisiones'],
        name='Comisiones',
        marker_color='red'
    ),
    row=2, col=1
)

# Actualizar diseño
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Análisis de Flujos de Dinero en Ekhilur 2024",
    barmode='relative'
)

fig.update_xaxes(title_text="Mes", row=2, col=1)
fig.update_yaxes(title_text="Euros (€)", row=1, col=1)
fig.update_yaxes(title_text="Euros (€)", row=2, col=1)

# Mostrar la gráfica
fig.show()

# Mostrar resumen estadístico
print("\nResumen Estadístico del Balance 2024:")
print("-" * 50)
print(f"Total Entradas: {df_balance['total_entradas'].sum():,.2f}€")
print(f"- Recargas: {df_balance['recarga_tarjeta'].sum():,.2f}€")
print(f"- Bonificaciones: {df_balance['bonificaciones'].sum():,.2f}€")
print(f"\nTotal Salidas: {df_balance['total_salidas'].sum():,.2f}€")
print(f"- Conversiones: {df_balance['conversion_euros'].sum():,.2f}€")
print(f"- Comisiones: {df_balance['comisiones'].sum():,.2f}€")
print(f"- Cuotas Mensuales: {df_balance['cuotas_mensuales'].sum():,.2f}€")
print(f"- Cuotas Variables: {df_balance['cuotas_variables'].sum():,.2f}€")
print(f"\nBalance Neto: {df_balance['balance_neto'].sum():,.2f}€")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Resumen Estadístico del Balance 2024:
--------------------------------------------------
Total Entradas: 1,408,848.49€
- Recargas: 1,255,837.00€
- Bonificaciones: 153,011.49€

Total Salidas: 1,300,375.57€
- Conversiones: 1,267,647.50€
- Comisiones: 14,838.46€
- Cuotas Mensuales: 11,760.45€
- Cuotas Variables: 6,129.16€

Balance Neto: 108,472.92€


### 4.5 Análisis de Bonificaciones del Ayuntamiento

Las bonificaciones municipales son una parte fundamental del circuito Ekhilur, representando la inversión directa del ayuntamiento para estimular la economía local. Este análisis se centra en:

1. **Control de Límites**
   - Límite global: 12,500€ mensuales del ayuntamiento
   - Límite individual: 100€ por usuario/mes
   - Caso especial: Agosto (remanente transferido a diciembre)

2. **Distribución de Bonificaciones**
   - Distribución mensual del fondo
   - Número de usuarios beneficiados
   - Promedio de bonificación por usuario
   - Distribución entre particulares y profesionales

3. **Impacto Económico**
   - Relación bonificaciones/compras totales
   - Efecto multiplicador (€ en compras por € bonificado)
   - Patrones de uso y efectividad

4. **Indicadores de Rendimiento**
   - Porcentaje de utilización del fondo mensual
   - Distribución entre sectores comerciales
   - Evolución temporal del programa

In [7]:
# Consulta SQL para análisis de bonificaciones
query_bonificaciones = """
WITH bonificaciones_mensuales AS (
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        CASE SUBSTRING(f.Id_fecha, 5, 2)
            WHEN '01' THEN 'Enero'
            WHEN '02' THEN 'Febrero'
            WHEN '03' THEN 'Marzo'
            WHEN '04' THEN 'Abril'
            WHEN '05' THEN 'Mayo'
            WHEN '06' THEN 'Junio'
            WHEN '07' THEN 'Julio'
            WHEN '08' THEN 'Agosto'
            WHEN '09' THEN 'Septiembre'
            WHEN '10' THEN 'Octubre'
            WHEN '11' THEN 'Noviembre'
            WHEN '12' THEN 'Diciembre'
        END as nombre_mes,
        -- Datos de bonificaciones
        SUM(f.Cantidad) as total_bonificaciones,
        COUNT(DISTINCT f.Usuario_receptor) as usuarios_beneficiados,
        SUM(f.Cantidad) / COUNT(DISTINCT f.Usuario_receptor) as promedio_por_usuario,
        -- Máxima bonificación por usuario
        MAX(f.Cantidad) as maxima_bonificacion,
        -- Conteo por tipo de usuario
        COUNT(DISTINCT CASE 
            WHEN u.Tipo_usuario = 'usuario' THEN f.Usuario_receptor 
        END) as usuarios_particulares,
        COUNT(DISTINCT CASE 
            WHEN u.Tipo_usuario IN ('autonomo', 'Empresa') THEN f.Usuario_receptor 
        END) as usuarios_empresas,
        -- Suma por tipo de usuario
        SUM(CASE 
            WHEN u.Tipo_usuario = 'usuario' THEN f.Cantidad 
            ELSE 0 
        END) as suma_particulares,
        SUM(CASE 
            WHEN u.Tipo_usuario IN ('autonomo', 'Empresa') THEN f.Cantidad 
            ELSE 0 
        END) as suma_empresas,
        -- Comparación con límite mensual
        12500 as limite_mensual,
        (SUM(f.Cantidad) / 12500) * 100 as porcentaje_uso
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    JOIN dim_usuarios u ON f.Usuario_receptor = u.Id_usuario
    WHERE o.Operacion = 'Bonificación por compra'
    AND SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    AND f.Usuario_emisor = 'dea65d4fd2a24ea5a7d621beab8d2c7889114849e374b53933bda0e8a1ade7a4'
    AND u.Tipo_usuario IN ('usuario', 'autonomo', 'Empresa')  -- Excluimos Asociacion y ekhilur
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6),
             CASE SUBSTRING(f.Id_fecha, 5, 2)
                WHEN '01' THEN 'Enero'
                WHEN '02' THEN 'Febrero'
                WHEN '03' THEN 'Marzo'
                WHEN '04' THEN 'Abril'
                WHEN '05' THEN 'Mayo'
                WHEN '06' THEN 'Junio'
                WHEN '07' THEN 'Julio'
                WHEN '08' THEN 'Agosto'
                WHEN '09' THEN 'Septiembre'
                WHEN '10' THEN 'Octubre'
                WHEN '11' THEN 'Noviembre'
                WHEN '12' THEN 'Diciembre'
             END
)
SELECT 
    nombre_mes,
    ROUND(total_bonificaciones, 2) as total_bonificaciones,
    usuarios_beneficiados,
    ROUND(promedio_por_usuario, 2) as promedio_por_usuario,
    ROUND(maxima_bonificacion, 2) as maxima_bonificacion,
    usuarios_particulares,
    usuarios_empresas,
    ROUND(suma_particulares, 2) as suma_particulares,
    ROUND(suma_empresas, 2) as suma_empresas,
    limite_mensual,
    ROUND(porcentaje_uso, 2) as porcentaje_uso
FROM bonificaciones_mensuales
ORDER BY año_mes;
"""

# Ejecutar la consulta
df_bonificaciones = ejecutar_query(query_bonificaciones)

# Crear visualizaciones
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Uso del Fondo Mensual de Bonificaciones',
        'Distribución por Tipo de Usuario',
        'Usuarios Beneficiados por Mes',
        'Promedio de Bonificación por Usuario'
    ),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# 1. Gráfico de barras: Uso del fondo mensual
fig.add_trace(
    go.Bar(
        name='Bonificaciones',
        x=df_bonificaciones['nombre_mes'],
        y=df_bonificaciones['total_bonificaciones'],
        marker_color='green'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        name='Límite Mensual',
        x=df_bonificaciones['nombre_mes'],
        y=df_bonificaciones['limite_mensual'],
        line=dict(color='red', dash='dash')
    ),
    row=1, col=1
)

# 2. Gráfico circular: Distribución total entre particulares y empresas
fig.add_trace(
    go.Pie(
        labels=['Particulares', 'Empresas'],
        values=[
            df_bonificaciones['suma_particulares'].sum(),
            df_bonificaciones['suma_empresas'].sum()
        ],
        marker_colors=['lightblue', 'darkblue']
    ),
    row=1, col=2
)

# 3. Gráfico de líneas: Usuarios beneficiados por mes
fig.add_trace(
    go.Scatter(
        name='Particulares',
        x=df_bonificaciones['nombre_mes'],
        y=df_bonificaciones['usuarios_particulares'],
        line=dict(color='lightblue')
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
        name='Empresas',
        x=df_bonificaciones['nombre_mes'],
        y=df_bonificaciones['usuarios_empresas'],
        line=dict(color='darkblue')
    ),
    row=2, col=1
)

# 4. Gráfico de barras: Promedio de bonificación por usuario
fig.add_trace(
    go.Bar(
        name='Promedio por Usuario',
        x=df_bonificaciones['nombre_mes'],
        y=df_bonificaciones['promedio_por_usuario'],
        marker_color='purple'
    ),
    row=2, col=2
)

# Actualizar diseño
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Análisis de Bonificaciones Municipales 2024"
)

# Actualizar etiquetas de los ejes
fig.update_yaxes(title_text="Euros (€)", row=1, col=1)
fig.update_yaxes(title_text="Número de Usuarios", row=2, col=1)
fig.update_yaxes(title_text="Euros (€)", row=2, col=2)

# Mostrar la gráfica
fig.show()

# Mostrar resumen estadístico
print("\nResumen Estadístico de Bonificaciones 2024:")
print("-" * 50)
print(f"Total Bonificaciones: {df_bonificaciones['total_bonificaciones'].sum():,.2f}€")
print(f"Total Usuarios Beneficiados: {df_bonificaciones['usuarios_beneficiados'].sum():,}")
print(f"\nDistribución por Tipo:")
print(f"- Particulares: {df_bonificaciones['suma_particulares'].sum():,.2f}€ ({df_bonificaciones['usuarios_particulares'].sum():,} usuarios)")
print(f"- Empresas: {df_bonificaciones['suma_empresas'].sum():,.2f}€ ({df_bonificaciones['usuarios_empresas'].sum():,} usuarios)")
print(f"\nPromedios:")
print(f"- Bonificación media por usuario: {df_bonificaciones['promedio_por_usuario'].mean():,.2f}€")
print(f"- Máxima bonificación registrada: {df_bonificaciones['maxima_bonificacion'].max():,.2f}€")
print(f"\nUso del Fondo:")
print(f"- Promedio mensual de uso: {df_bonificaciones['porcentaje_uso'].mean():,.2f}%")
print(f"- Mes con mayor uso: {df_bonificaciones.loc[df_bonificaciones['porcentaje_uso'].idxmax(), 'nombre_mes']} ({df_bonificaciones['porcentaje_uso'].max():,.2f}%)")
print(f"- Mes con menor uso: {df_bonificaciones.loc[df_bonificaciones['porcentaje_uso'].idxmin(), 'nombre_mes']} ({df_bonificaciones['porcentaje_uso'].min():,.2f}%)")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Resumen Estadístico de Bonificaciones 2024:
--------------------------------------------------
Total Bonificaciones: 152,662.20€
Total Usuarios Beneficiados: 10,398

Distribución por Tipo:
- Particulares: 140,290.30€ (9,571 usuarios)
- Empresas: 12,371.90€ (827 usuarios)

Promedios:
- Bonificación media por usuario: 14.65€
- Máxima bonificación registrada: 100.00€

Uso del Fondo:
- Promedio mensual de uso: 101.77%
- Mes con mayor uso: Diciembre (128.69%)
- Mes con menor uso: Agosto (80.52%)


### 4.6 Impacto de las Bonificaciones en las Ventas

Analizamos la efectividad del programa de bonificaciones estudiando:

1. **Relación Bonificación-Venta**
   - Volumen de ventas por cada euro bonificado
   - Distribución temporal de las ventas en relación con las bonificaciones
   - Patrones de compra de usuarios bonificados

2. **Impacto en Comercios**
   - Ventas en comercios que participan en el programa
   - Comparativa de ventas con y sin bonificación
   - Fidelización de clientes

3. **Eficiencia del Programa**
   - Retorno de inversión para el ayuntamiento
   - Distribución del impacto entre sectores comerciales
   - Efectividad en la estimulación del comercio local

In [8]:
# Consulta SQL para analizar la relación entre bonificaciones y ventas
query_impacto_ventas = """
WITH ventas_comercios AS (
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        CASE SUBSTRING(f.Id_fecha, 5, 2)
            WHEN '01' THEN 'Enero'
            WHEN '02' THEN 'Febrero'
            WHEN '03' THEN 'Marzo'
            WHEN '04' THEN 'Abril'
            WHEN '05' THEN 'Mayo'
            WHEN '06' THEN 'Junio'
            WHEN '07' THEN 'Julio'
            WHEN '08' THEN 'Agosto'
            WHEN '09' THEN 'Septiembre'
            WHEN '10' THEN 'Octubre'
            WHEN '11' THEN 'Noviembre'
            WHEN '12' THEN 'Diciembre'
        END as nombre_mes,
        -- Ventas totales (solo compras normales)
        SUM(CASE 
            WHEN o.Operacion IN ('Pago a usuario', 'Cobro desde QR') 
            AND (
                ue.Tipo_usuario IN ('usuario', 'autonomo', 'Empresa')
                AND ur.Tipo_usuario IN ('autonomo', 'Empresa')
            )
            THEN f.Cantidad 
            ELSE 0 
        END) as ventas_totales,
        -- Bonificaciones
        SUM(CASE 
            WHEN o.Operacion = 'Bonificación por compra' 
            THEN f.Cantidad 
            ELSE 0 
        END) as bonificaciones,
        -- Número de transacciones
        COUNT(DISTINCT CASE 
            WHEN o.Operacion IN ('Pago a usuario', 'Cobro desde QR')
            AND (
                ue.Tipo_usuario IN ('usuario', 'autonomo', 'Empresa')
                AND ur.Tipo_usuario IN ('autonomo', 'Empresa')
            )
            THEN f.Id_transaccion 
        END) as num_ventas,
        -- Número de comercios con ventas
        COUNT(DISTINCT CASE 
            WHEN o.Operacion IN ('Pago a usuario', 'Cobro desde QR')
            AND ur.Tipo_usuario IN ('autonomo', 'Empresa')
            THEN f.Usuario_receptor 
        END) as comercios_activos,
        -- Número de compradores únicos (al menos una compra en el mes)
        COUNT(DISTINCT CASE 
            WHEN o.Operacion IN ('Pago a usuario', 'Cobro desde QR')
            AND (
                ue.Tipo_usuario IN ('usuario', 'autonomo', 'Empresa')
                AND ur.Tipo_usuario IN ('autonomo', 'Empresa')
            )
            THEN f.Usuario_emisor 
        END) as compradores_unicos
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    JOIN dim_usuarios ur ON f.Usuario_receptor = ur.Id_usuario
    JOIN dim_usuarios ue ON f.Usuario_emisor = ue.Id_usuario
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6),
             CASE SUBSTRING(f.Id_fecha, 5, 2)
                WHEN '01' THEN 'Enero'
                WHEN '02' THEN 'Febrero'
                WHEN '03' THEN 'Marzo'
                WHEN '04' THEN 'Abril'
                WHEN '05' THEN 'Mayo'
                WHEN '06' THEN 'Junio'
                WHEN '07' THEN 'Julio'
                WHEN '08' THEN 'Agosto'
                WHEN '09' THEN 'Septiembre'
                WHEN '10' THEN 'Octubre'
                WHEN '11' THEN 'Noviembre'
                WHEN '12' THEN 'Diciembre'
             END
)
SELECT 
    nombre_mes,
    ROUND(ventas_totales, 2) as ventas_totales,
    ROUND(bonificaciones, 2) as bonificaciones,
    num_ventas,
    comercios_activos,
    compradores_unicos,
    ROUND(ventas_totales / NULLIF(bonificaciones, 0), 2) as multiplicador_bonificacion,
    ROUND(ventas_totales / comercios_activos, 2) as venta_media_por_comercio,
    ROUND(ventas_totales / num_ventas, 2) as ticket_medio
FROM ventas_comercios
ORDER BY año_mes;
"""

# Ejecutar la consulta
df_impacto = ejecutar_query(query_impacto_ventas)

# Crear visualizaciones
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Ventas vs Bonificaciones por Mes',
        'Efecto Multiplicador de Bonificaciones',
        'Actividad Comercial',
        'Ticket Medio y Venta por Comercio'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# 1. Gráfico de barras: Ventas vs Bonificaciones
fig.add_trace(
    go.Bar(
        name='Ventas Totales',
        x=df_impacto['nombre_mes'],
        y=df_impacto['ventas_totales'],
        marker_color='blue'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        name='Bonificaciones',
        x=df_impacto['nombre_mes'],
        y=df_impacto['bonificaciones'],
        marker_color='green'
    ),
    row=1, col=1
)

# 2. Gráfico de líneas: Multiplicador de bonificaciones
fig.add_trace(
    go.Scatter(
        name='Multiplicador',
        x=df_impacto['nombre_mes'],
        y=df_impacto['multiplicador_bonificacion'],
        mode='lines+markers',
        line=dict(color='red')
    ),
    row=1, col=2
)

# 3. Gráfico de líneas: Actividad comercial
fig.add_trace(
    go.Scatter(
        name='Comercios Activos',
        x=df_impacto['nombre_mes'],
        y=df_impacto['comercios_activos'],
        mode='lines+markers',
        line=dict(color='purple')
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
        name='Compradores Únicos',
        x=df_impacto['nombre_mes'],
        y=df_impacto['compradores_unicos'],
        mode='lines+markers',
        line=dict(color='orange')
    ),
    row=2, col=1
)

# 4. Gráfico de barras: Ticket medio y venta por comercio
fig.add_trace(
    go.Bar(
        name='Ticket Medio',
        x=df_impacto['nombre_mes'],
        y=df_impacto['ticket_medio'],
        marker_color='lightblue'
    ),
    row=2, col=2
)
fig.add_trace(
    go.Bar(
        name='Venta Media por Comercio',
        x=df_impacto['nombre_mes'],
        y=df_impacto['venta_media_por_comercio'],
        marker_color='darkblue'
    ),
    row=2, col=2
)

# Actualizar diseño
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Impacto de las Bonificaciones en las Ventas 2024",
    barmode='group'
)

# Actualizar etiquetas de los ejes
fig.update_yaxes(title_text="Euros (€)", row=1, col=1)
fig.update_yaxes(title_text="Ratio Venta/Bonificación", row=1, col=2)
fig.update_yaxes(title_text="Número", row=2, col=1)
fig.update_yaxes(title_text="Euros (€)", row=2, col=2)

# Mostrar la gráfica
fig.show()

# Mostrar resumen estadístico
print("\nResumen del Impacto de Bonificaciones en Ventas 2024:")
print("-" * 50)
print(f"Ventas Totales: {df_impacto['ventas_totales'].sum():,.2f}€")
print(f"Total Bonificaciones: {df_impacto['bonificaciones'].sum():,.2f}€")
print(f"Multiplicador Medio: {df_impacto['multiplicador_bonificacion'].mean():,.2f}x")
print(f"\nActividad Comercial:")
print(f"- Número total de ventas: {df_impacto['num_ventas'].sum():,}")
print(f"- Promedio mensual de comercios activos: {df_impacto['comercios_activos'].mean():,.0f}")
print(f"- Promedio mensual de compradores únicos: {df_impacto['compradores_unicos'].mean():,.0f}")
print(f"\nPromedios:")
print(f"- Ticket medio: {df_impacto['ticket_medio'].mean():,.2f}€")
print(f"- Venta media mensual por comercio: {df_impacto['venta_media_por_comercio'].mean():,.2f}€")
print(f"\nMeses Destacados:")
print(f"- Mayor multiplicador: {df_impacto.loc[df_impacto['multiplicador_bonificacion'].idxmax(), 'nombre_mes']} ({df_impacto['multiplicador_bonificacion'].max():,.2f}x)")
print(f"- Mayor volumen de ventas: {df_impacto.loc[df_impacto['ventas_totales'].idxmax(), 'nombre_mes']} ({df_impacto['ventas_totales'].max():,.2f}€)")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Resumen del Impacto de Bonificaciones en Ventas 2024:
--------------------------------------------------
Ventas Totales: 2,504,894.68€
Total Bonificaciones: 153,008.00€
Multiplicador Medio: 16.32x

Actividad Comercial:
- Número total de ventas: 105,677
- Promedio mensual de comercios activos: 96
- Promedio mensual de compradores únicos: 868

Promedios:
- Ticket medio: 23.67€
- Venta media mensual por comercio: 2,165.30€

Meses Destacados:
- Mayor multiplicador: Mayo (17.72x)
- Mayor volumen de ventas: Diciembre (281,357.17€)


### 4.7 Análisis de Efectividad de Descuentos Automáticos

Analizaremos el impacto real de los descuentos en el comportamiento de compra, comparando los patrones de los usuarios 3 meses antes y después de empezar a usar descuentos. El análisis se estructura en tres áreas clave:

1. **Patrones de Compra**
   
   Examinaremos:
   - Distribución de usuarios por frecuencia de compra (≤1, 1-5, 5-15, 15-30, >30 compras/mes)
   - Evolución del número de días de compra mensual
   - Cambios en el ticket medio por compra
   - Transformación de usuarios ocasionales a regulares

2. **Impacto en Ventas**

   Analizaremos:
   - Segmentación de usuarios por nivel de gasto mensual (≤50€, 50-150€, 150-500€, 500-1000€, >1000€)
   - Cambios en el gasto mensual promedio
   - Evolución de la distribución del gasto
   - Identificación de segmentos de mayor crecimiento

3. **Visualización de Resultados**

   Presentaremos:
   - Gráficos de barras comparativos antes/después para frecuencia y gasto
   - Gráficos circulares mostrando la distribución actual
   - Estadísticas detalladas de cada segmento
   - Porcentajes de cambio en cada categoría

Este análisis nos permitirá evaluar la efectividad del programa de descuentos y entender cómo ha influido en los hábitos de compra de los usuarios.

In [9]:
query_distribucion = """
WITH primera_fecha_descuento AS (
    SELECT 
        f.Usuario_receptor,
        MIN(f.Id_fecha) as primera_fecha_descuento
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE o.Operacion = 'Descuento automático'
    GROUP BY f.Usuario_receptor
),
metricas_usuario_periodo AS (
    SELECT 
        f.Usuario_emisor,
        CASE 
            WHEN f.Id_fecha < fd.primera_fecha_descuento THEN 'Antes'
            ELSE 'Después'
        END as periodo,
        COUNT(*) / 3.0 as compras_mensuales,
        SUM(f.Cantidad) / 3.0 as gasto_mensual
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    JOIN primera_fecha_descuento fd ON f.Usuario_emisor = fd.Usuario_receptor
    WHERE o.Operacion IN ('Pago a usuario', 'Cobro desde QR')
    AND (
        f.Id_fecha BETWEEN 
            DATE_FORMAT(DATE_SUB(STR_TO_DATE(fd.primera_fecha_descuento, '%Y%m%d'), INTERVAL 90 DAY), '%Y%m%d')
            AND fd.primera_fecha_descuento
        OR 
        f.Id_fecha BETWEEN 
            fd.primera_fecha_descuento 
            AND DATE_FORMAT(DATE_ADD(STR_TO_DATE(fd.primera_fecha_descuento, '%Y%m%d'), INTERVAL 90 DAY), '%Y%m%d')
    )
    GROUP BY 
        f.Usuario_emisor,
        CASE 
            WHEN f.Id_fecha < fd.primera_fecha_descuento THEN 'Antes'
            ELSE 'Después'
        END
)
SELECT 
    periodo,
    -- Distribución por frecuencia de compras
    SUM(CASE WHEN compras_mensuales <= 1 THEN 1 ELSE 0 END) as compras_muy_bajo,
    SUM(CASE WHEN compras_mensuales > 1 AND compras_mensuales <= 5 THEN 1 ELSE 0 END) as compras_bajo,
    SUM(CASE WHEN compras_mensuales > 5 AND compras_mensuales <= 15 THEN 1 ELSE 0 END) as compras_medio,
    SUM(CASE WHEN compras_mensuales > 15 AND compras_mensuales <= 30 THEN 1 ELSE 0 END) as compras_alto,
    SUM(CASE WHEN compras_mensuales > 30 THEN 1 ELSE 0 END) as compras_muy_alto,
    -- Distribución por gasto mensual
    SUM(CASE WHEN gasto_mensual <= 50 THEN 1 ELSE 0 END) as gasto_muy_bajo,
    SUM(CASE WHEN gasto_mensual > 50 AND gasto_mensual <= 150 THEN 1 ELSE 0 END) as gasto_bajo,
    SUM(CASE WHEN gasto_mensual > 150 AND gasto_mensual <= 500 THEN 1 ELSE 0 END) as gasto_medio,
    SUM(CASE WHEN gasto_mensual > 500 AND gasto_mensual <= 1000 THEN 1 ELSE 0 END) as gasto_alto,
    SUM(CASE WHEN gasto_mensual > 1000 THEN 1 ELSE 0 END) as gasto_muy_alto,
    COUNT(*) as total_usuarios
FROM metricas_usuario_periodo
GROUP BY periodo
ORDER BY periodo;
"""

# Ejecutar la consulta
df_distribucion = ejecutar_query(query_distribucion)

# Crear figura con subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Distribución de Frecuencia de Compras',
        'Cambio en Patrones de Compra',
        'Distribución de Gasto Mensual',
        'Cambio en Patrones de Gasto'
    ),
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "pie"}]]
)

# 1. Gráfico de barras para frecuencia de compras
categorias_compras = ['≤1 compra/mes', '1-5 compras/mes', '5-15 compras/mes', '15-30 compras/mes', '>30 compras/mes']
valores_antes = [
    df_distribucion.iloc[0]['compras_muy_bajo'],
    df_distribucion.iloc[0]['compras_bajo'],
    df_distribucion.iloc[0]['compras_medio'],
    df_distribucion.iloc[0]['compras_alto'],
    df_distribucion.iloc[0]['compras_muy_alto']
]
valores_despues = [
    df_distribucion.iloc[1]['compras_muy_bajo'],
    df_distribucion.iloc[1]['compras_bajo'],
    df_distribucion.iloc[1]['compras_medio'],
    df_distribucion.iloc[1]['compras_alto'],
    df_distribucion.iloc[1]['compras_muy_alto']
]

fig.add_trace(
    go.Bar(name='Antes', x=categorias_compras, y=valores_antes, marker_color='lightcoral'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='Después', x=categorias_compras, y=valores_despues, marker_color='lightgreen'),
    row=1, col=1
)

# 2. Gráfico circular para el cambio en compras
fig.add_trace(
    go.Pie(
        labels=categorias_compras,
        values=valores_despues,
        hole=.3,
        marker_colors=['lightcoral', 'lightsalmon', 'lightgreen', 'mediumseagreen', 'darkgreen']
    ),
    row=1, col=2
)

# 3. Gráfico de barras para gasto mensual
categorias_gasto = ['≤50€/mes', '50-150€/mes', '150-500€/mes', '500-1000€/mes', '>1000€/mes']
valores_gasto_antes = [
    df_distribucion.iloc[0]['gasto_muy_bajo'],
    df_distribucion.iloc[0]['gasto_bajo'],
    df_distribucion.iloc[0]['gasto_medio'],
    df_distribucion.iloc[0]['gasto_alto'],
    df_distribucion.iloc[0]['gasto_muy_alto']
]
valores_gasto_despues = [
    df_distribucion.iloc[1]['gasto_muy_bajo'],
    df_distribucion.iloc[1]['gasto_bajo'],
    df_distribucion.iloc[1]['gasto_medio'],
    df_distribucion.iloc[1]['gasto_alto'],
    df_distribucion.iloc[1]['gasto_muy_alto']
]

fig.add_trace(
    go.Bar(name='Antes', x=categorias_gasto, y=valores_gasto_antes, marker_color='lightcoral'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(name='Después', x=categorias_gasto, y=valores_gasto_despues, marker_color='lightgreen'),
    row=2, col=1
)

# 4. Gráfico circular para el cambio en gasto
fig.add_trace(
    go.Pie(
        labels=categorias_gasto,
        values=valores_gasto_despues,
        hole=.3,
        marker_colors=['lightcoral', 'lightsalmon', 'lightgreen', 'mediumseagreen', 'darkgreen']
    ),
    row=2, col=2
)

# Actualizar diseño
fig.update_layout(
    height=1000,
    showlegend=True,
    title_text="Impacto de los Descuentos en Patrones de Uso",
)

# Actualizar ejes
fig.update_xaxes(title_text="Categoría de Compras", row=1, col=1)
fig.update_xaxes(title_text="Categoría de Gasto", row=2, col=1)
fig.update_yaxes(title_text="Número de Usuarios", row=1, col=1)
fig.update_yaxes(title_text="Número de Usuarios", row=2, col=1)

# Mostrar la figura
fig.show()

# Imprimir estadísticas
print("\nEstadísticas Detalladas del Impacto de Descuentos")
print("-" * 70)
for index, row in df_distribucion.iterrows():
    total = row['total_usuarios']
    periodo = row['periodo']
    
    print(f"\n{periodo.upper()} DE DESCUENTOS:")
    print("\nFrecuencia de Compras:")
    for cat, val in zip(categorias_compras, [
        row['compras_muy_bajo'],
        row['compras_bajo'],
        row['compras_medio'],
        row['compras_alto'],
        row['compras_muy_alto']
    ]):
        print(f"- {cat}: {val:.0f} usuarios ({val/total*100:.1f}%)")
    
    print("\nGasto Mensual:")
    for cat, val in zip(categorias_gasto, [
        row['gasto_muy_bajo'],
        row['gasto_bajo'],
        row['gasto_medio'],
        row['gasto_alto'],
        row['gasto_muy_alto']
    ]):
        print(f"- {cat}: {val:.0f} usuarios ({val/total*100:.1f}%)")
    
    print(f"\nTotal usuarios: {total}")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Estadísticas Detalladas del Impacto de Descuentos
----------------------------------------------------------------------

ANTES DE DESCUENTOS:

Frecuencia de Compras:
- ≤1 compra/mes: 156 usuarios (37.3%)
- 1-5 compras/mes: 159 usuarios (38.0%)
- 5-15 compras/mes: 81 usuarios (19.4%)
- 15-30 compras/mes: 13 usuarios (3.1%)
- >30 compras/mes: 9 usuarios (2.2%)

Gasto Mensual:
- ≤50€/mes: 215 usuarios (51.4%)
- 50-150€/mes: 120 usuarios (28.7%)
- 150-500€/mes: 69 usuarios (16.5%)
- 500-1000€/mes: 10 usuarios (2.4%)
- >1000€/mes: 4 usuarios (1.0%)

Total usuarios: 418

DESPUÉS DE DESCUENTOS:

Frecuencia de Compras:
- ≤1 compra/mes: 31 usuarios (4.5%)
- 1-5 compras/mes: 166 usuarios (23.9%)
- 5-15 compras/mes: 285 usuarios (40.9%)
- 15-30 compras/mes: 143 usuarios (20.5%)
- >30 compras/mes: 71 usuarios (10.2%)

Gasto Mensual:
- ≤50€/mes: 50 usuarios (7.2%)
- 50-150€/mes: 163 usuarios (23.4%)
- 150-500€/mes: 349 usuarios (50.1%)
- 500-1000€/mes: 104 usuarios (14.9%)
- >1000€/mes: 30 usuari

### 4.8 Análisis de Ingresos de Ekhilur

Analizaremos los cuatro tipos de ingresos directos que tiene Ekhilur en 2024, que suman un total de 39,238.07€:

1. **Comisiones por Retirada** (14,838.46€ - 37.8%)
   - 967 operaciones de 95 usuarios únicos
   - Promedio: 15.34€ por operación
   - Por tipo de usuario:
     * Empresas (41): 8,547.27€ (208.47€/empresa)
     * Autónomos (44): 5,997.31€ (136.30€/autónomo)
     * Otros (10): 293.88€ (29.39€/usuario)

2. **Cuotas Mensuales** (11,760.45€ - 30.0%)
   - 6,031 cuotas de 627 usuarios únicos
   - Cuota fija: 1.95€ mensual
   - Por tipo de usuario:
     * Usuarios (480): 11,514.75€ (97.9%)
     * Autónomos (9): 171.60€ (1.5%)
     * Otros (4): 74.10€ (0.6%)

3. **Cuotas de Socio** (6,510.00€ - 16.6%)
   - 651 nuevos socios en 2024
   - Cuota única: 10.00€
   - Principal fuente de crecimiento de la base de usuarios

4. **Cuotas Variables** (6,129.16€ - 15.6%)
   - 1,512 operaciones de 143 usuarios únicos
   - Promedio: 4.05€ por operación
   - Por tipo de usuario:
     * Empresas (60): 3,185.24€ (4.86€/mes)
     * Autónomos (63): 2,533.84€ (3.90€/mes)
     * Asociaciones (20): 410.08€ (1.98€/mes)

**Métricas Clave:**
- Total operaciones: 9,161
- Base de usuarios activa: 627
- ARPU (Ingreso medio por usuario): 62.58€
- Tasa de conversión a socios: 92.8% (651/702 usuarios)

**Conclusiones:**
1. Las empresas y autónomos generan el 98% de las comisiones por retirada
2. Los usuarios particulares dominan las cuotas mensuales (97.9%)
3. Fuerte crecimiento en nuevos socios (651 en 2024)
4. Las cuotas variables se concentran en empresas y autónomos (93.3%)

In [10]:
query_ingresos_mensual = """
WITH ingresos_mensuales AS (
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        o.Operacion,
        SUM(f.Cantidad) as total_cantidad
    FROM fact_table f
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE o.Operacion IN (
        'Comisión por retirada',
        'Cuota de socio',
        'Cuota mensual de socio',
        'Cuota variable'
    )
    AND SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    GROUP BY 
        SUBSTRING(f.Id_fecha, 1, 6),
        o.Operacion
)
SELECT 
    año_mes,
    CASE SUBSTRING(año_mes, 5, 2)
        WHEN '01' THEN 'Enero'
        WHEN '02' THEN 'Febrero'
        WHEN '03' THEN 'Marzo'
        WHEN '04' THEN 'Abril'
        WHEN '05' THEN 'Mayo'
        WHEN '06' THEN 'Junio'
        WHEN '07' THEN 'Julio'
        WHEN '08' THEN 'Agosto'
        WHEN '09' THEN 'Septiembre'
        WHEN '10' THEN 'Octubre'
        WHEN '11' THEN 'Noviembre'
        WHEN '12' THEN 'Diciembre'
    END as mes,
    Operacion,
    ROUND(total_cantidad, 2) as total_euros
FROM ingresos_mensuales
ORDER BY año_mes, Operacion;
"""

# Ejecutar la consulta
df_ingresos = ejecutar_query(query_ingresos_mensual)

# Crear figura
fig = go.Figure()

# Calcular totales mensuales
totales_mensuales = df_ingresos.groupby(['año_mes', 'mes'])['total_euros'].sum().reset_index()
totales_mensuales = totales_mensuales.sort_values('año_mes')

# Añadir línea de total
fig.add_trace(
    go.Scatter(
        name='Total',
        x=totales_mensuales['mes'],
        y=totales_mensuales['total_euros'],
        mode='lines+markers',
        line=dict(width=3, dash='solid'),
        marker=dict(size=10),
    )
)

# Gráfico de líneas para cada tipo de ingreso
for operacion in df_ingresos['Operacion'].unique():
    datos = df_ingresos[df_ingresos['Operacion'] == operacion]
    datos_ordenados = datos.sort_values('año_mes')
    fig.add_trace(
        go.Scatter(
            name=operacion,
            x=datos_ordenados['mes'],
            y=datos_ordenados['total_euros'],
            mode='lines+markers',
        )
    )

# Actualizar diseño
fig.update_layout(
    height=600,
    title_text="Evolución de Ingresos Ekhilur 2024",
    xaxis_title="Mes",
    yaxis_title="Euros (€)",
    showlegend=True,
)

# Mostrar la figura
fig.show()

# Imprimir resumen mensual
print("\nResumen Mensual de Ingresos")
print("-" * 50)
df_ingresos_ordenado = df_ingresos.sort_values('año_mes')
for mes, datos_mes in df_ingresos_ordenado.groupby(['año_mes', 'mes']):
    total_mes = datos_mes['total_euros'].sum()
    print(f"\n{mes[1]}: {total_mes:,.2f}€")
    for _, fila in datos_mes.iterrows():
        porcentaje = (fila['total_euros'] / total_mes) * 100
        print(f"  - {fila['Operacion']}: {fila['total_euros']:,.2f}€ ({porcentaje:.1f}%)")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Resumen Mensual de Ingresos
--------------------------------------------------

Enero: 4,953.16€
  - Comisión por retirada: 1,050.75€ (21.2%)
  - Cuota de socio: 3,270.00€ (66.0%)
  - Cuota mensual de socio: 120.90€ (2.4%)
  - Cuota variable: 511.51€ (10.3%)

Febrero: 3,180.45€
  - Comisión por retirada: 1,222.93€ (38.5%)
  - Cuota de socio: 720.00€ (22.6%)
  - Cuota mensual de socio: 696.15€ (21.9%)
  - Cuota variable: 541.37€ (17.0%)

Marzo: 2,666.05€
  - Cuota variable: 531.02€ (19.9%)
  - Cuota mensual de socio: 846.30€ (31.7%)
  - Comisión por retirada: 908.73€ (34.1%)
  - Cuota de socio: 380.00€ (14.3%)

Abril: 3,186.38€
  - Comisión por retirada: 1,321.60€ (41.5%)
  - Cuota de socio: 430.00€ (13.5%)
  - Cuota mensual de socio: 908.70€ (28.5%)
  - Cuota variable: 526.08€ (16.5%)

Mayo: 3,236.90€
  - Cuota variable: 545.75€ (16.9%)
  - Comisión por retirada: 1,284.45€ (39.7%)
  - Cuota de socio: 420.00€ (13.0%)
  - Cuota mensual de socio: 986.70€ (30.5%)

Junio: 3,102.04€
  - Cuo

In [11]:
query_tipos_usuario = """
SELECT DISTINCT Tipo_usuario
FROM dim_usuarios
ORDER BY Tipo_usuario;
"""

# Ejecutar la consulta
df_tipos = ejecutar_query(query_tipos_usuario)
print("Tipos de usuarios en la base de datos:")
print("-" * 35)
for tipo in df_tipos['Tipo_usuario']:
    print(f"- {tipo}")

Tipos de usuarios en la base de datos:
-----------------------------------
- Asociacion
- autonomo
- ekhilur
- Empresa
- usuario


C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [12]:
query_usuarios_tipo = """
WITH usuarios_por_periodo AS (
    -- Usuarios únicos por día y tipo
    SELECT 
        f.Id_fecha,
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        du.Tipo_usuario,
        COUNT(DISTINCT f.Usuario_emisor) as usuarios_unicos_dia
    FROM fact_table f
    JOIN dim_usuarios du ON f.Usuario_emisor = du.Id_usuario
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    AND du.Tipo_usuario IN ('usuario', 'Empresa', 'autonomo')  -- Solo estos tres tipos
    GROUP BY f.Id_fecha, SUBSTRING(f.Id_fecha, 1, 6), du.Tipo_usuario
),
usuarios_mensuales AS (
    -- Usuarios únicos por mes y tipo
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        du.Tipo_usuario,
        COUNT(DISTINCT f.Usuario_emisor) as usuarios_unicos_mes
    FROM fact_table f
    JOIN dim_usuarios du ON f.Usuario_emisor = du.Id_usuario
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    AND du.Tipo_usuario IN ('usuario', 'Empresa', 'autonomo')  -- Solo estos tres tipos
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6), du.Tipo_usuario
)
SELECT 
    u.Id_fecha,
    u.año_mes,
    CASE SUBSTRING(u.año_mes, 5, 2)
        WHEN '01' THEN 'Enero'
        WHEN '02' THEN 'Febrero'
        WHEN '03' THEN 'Marzo'
        WHEN '04' THEN 'Abril'
        WHEN '05' THEN 'Mayo'
        WHEN '06' THEN 'Junio'
        WHEN '07' THEN 'Julio'
        WHEN '08' THEN 'Agosto'
        WHEN '09' THEN 'Septiembre'
        WHEN '10' THEN 'Octubre'
        WHEN '11' THEN 'Noviembre'
        WHEN '12' THEN 'Diciembre'
    END as mes,
    u.Tipo_usuario,
    u.usuarios_unicos_dia,
    m.usuarios_unicos_mes
FROM usuarios_por_periodo u
JOIN usuarios_mensuales m ON u.año_mes = m.año_mes AND u.Tipo_usuario = m.Tipo_usuario
ORDER BY u.Id_fecha, u.Tipo_usuario;
"""

# Ejecutar la consulta
df_usuarios = ejecutar_query(query_usuarios_tipo)

# Crear tres subplots separados
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Usuarios Particulares',
        'Autónomos',
        'Empresas'
    ),
    vertical_spacing=0.12,
    row_heights=[0.33, 0.33, 0.33]
)

# Colores para cada tipo de usuario
colores = {
    'usuario': 'rgb(31, 119, 180)',    # Azul
    'autonomo': 'rgb(44, 160, 44)',    # Verde
    'Empresa': 'rgb(255, 127, 14)'     # Naranja
}

# 1. Gráfico para usuarios particulares
datos_usuarios = df_usuarios[df_usuarios['Tipo_usuario'] == 'usuario']
fig.add_trace(
    go.Scatter(
        x=datos_usuarios['mes'],
        y=datos_usuarios['usuarios_unicos_mes'],
        mode='lines+markers',
        name='Usuarios Particulares',
        line=dict(color=colores['usuario'], width=2),
        showlegend=True
    ),
    row=1, col=1
)

# 2. Gráfico para autónomos
datos_autonomos = df_usuarios[df_usuarios['Tipo_usuario'] == 'autonomo']
fig.add_trace(
    go.Scatter(
        x=datos_autonomos['mes'],
        y=datos_autonomos['usuarios_unicos_mes'],
        mode='lines+markers',
        name='Autónomos',
        line=dict(color=colores['autonomo'], width=2),
        showlegend=True
    ),
    row=2, col=1
)

# 3. Gráfico para empresas
datos_empresas = df_usuarios[df_usuarios['Tipo_usuario'] == 'Empresa']
fig.add_trace(
    go.Scatter(
        x=datos_empresas['mes'],
        y=datos_empresas['usuarios_unicos_mes'],
        mode='lines+markers',
        name='Empresas',
        line=dict(color=colores['Empresa'], width=2),
        showlegend=True
    ),
    row=3, col=1
)

# Actualizar diseño
fig.update_layout(
    height=900,
    title_text='Evolución de Usuarios Únicos por Tipo en Ekhilur 2024',
    showlegend=True
)

# Actualizar ejes
for i in range(1, 4):
    fig.update_xaxes(title_text="Mes", row=i, col=1)
    fig.update_yaxes(title_text="Número de Usuarios", row=i, col=1)

# Mostrar la figura
fig.show()

# Imprimir estadísticas
print("\nEstadísticas de Usuarios Únicos por Tipo")
print("-" * 50)

# Función para calcular el crecimiento
def calcular_crecimiento(datos):
    primer_mes = datos.iloc[0]
    ultimo_mes = datos.iloc[-1]
    crecimiento_abs = ultimo_mes - primer_mes
    crecimiento_porc = (crecimiento_abs / primer_mes) * 100
    return crecimiento_abs, crecimiento_porc

# Estadísticas por tipo de usuario
for tipo in ['usuario', 'autonomo', 'Empresa']:
    datos_tipo = df_usuarios[df_usuarios['Tipo_usuario'] == tipo]
    datos_mensuales = datos_tipo.groupby('mes')['usuarios_unicos_mes'].first()
    
    crecimiento_abs, crecimiento_porc = calcular_crecimiento(datos_mensuales)
    
    print(f"\n{tipo.capitalize()}:")
    print(f"- Usuarios iniciales (Enero): {datos_mensuales.iloc[0]}")
    print(f"- Usuarios finales (Diciembre): {datos_mensuales.iloc[-1]}")
    print(f"- Crecimiento absoluto: {crecimiento_abs:+.0f} usuarios")
    print(f"- Crecimiento porcentual: {crecimiento_porc:+.1f}%")
    print(f"- Promedio diario: {datos_tipo['usuarios_unicos_dia'].mean():.1f} usuarios")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




Estadísticas de Usuarios Únicos por Tipo
--------------------------------------------------

Usuario:
- Usuarios iniciales (Enero): 1175
- Usuarios finales (Diciembre): 1240
- Crecimiento absoluto: +65 usuarios
- Crecimiento porcentual: +5.5%
- Promedio diario: 226.7 usuarios

Autonomo:
- Usuarios iniciales (Enero): 61
- Usuarios finales (Diciembre): 63
- Crecimiento absoluto: +2 usuarios
- Crecimiento porcentual: +3.3%
- Promedio diario: 31.5 usuarios

Empresa:
- Usuarios iniciales (Enero): 54
- Usuarios finales (Diciembre): 58
- Crecimiento absoluto: +4 usuarios
- Crecimiento porcentual: +7.4%
- Promedio diario: 18.9 usuarios


In [13]:
query_actividad = """
WITH actividad_diaria AS (
    -- Actividad diaria: transacciones por usuario
    SELECT 
        f.Id_fecha,
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        du.Tipo_usuario,
        o.Operacion,
        COUNT(DISTINCT f.Usuario_emisor) as usuarios_unicos,
        COUNT(*) as num_transacciones,
        ROUND(CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT f.Usuario_emisor), 2) as transacciones_por_usuario
    FROM fact_table f
    JOIN dim_usuarios du ON f.Usuario_emisor = du.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    AND du.Tipo_usuario IN ('usuario', 'Empresa', 'autonomo')
    AND o.Operacion IN (
        'Retirada',
        'Recarga',
        'Pago',
        'Cobro'
    )
    GROUP BY f.Id_fecha, SUBSTRING(f.Id_fecha, 1, 6), du.Tipo_usuario, o.Operacion
),
actividad_mensual AS (
    -- Actividad mensual: transacciones por usuario
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        du.Tipo_usuario,
        o.Operacion,
        COUNT(DISTINCT f.Usuario_emisor) as usuarios_unicos_mes,
        COUNT(*) as num_transacciones_mes,
        ROUND(CAST(COUNT(*) AS FLOAT) / COUNT(DISTINCT f.Usuario_emisor), 2) as transacciones_por_usuario_mes
    FROM fact_table f
    JOIN dim_usuarios du ON f.Usuario_emisor = du.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    AND du.Tipo_usuario IN ('usuario', 'Empresa', 'autonomo')
    AND o.Operacion IN (
        'Retirada',
        'Recarga',
        'Pago',
        'Cobro'
    )
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6), du.Tipo_usuario, o.Operacion
)
SELECT DISTINCT o.Operacion
FROM dim_operaciones o
ORDER BY o.Operacion;
"""

# Ejecutar la consulta para ver qué operaciones existen
df_operaciones = ejecutar_query(query_actividad)
print("Operaciones disponibles en la base de datos:")
print("-" * 40)
for op in df_operaciones['Operacion']:
    print(f"- {op}")

Operaciones disponibles en la base de datos:
----------------------------------------
- Bonificación por compra
- Cobro desde QR
- Comisión por retirada
- Conversión a €
- Cuota de socio
- Cuota mensual de socio
- Cuota variable
- Descuento automático
- Donación
- Pago a usuario
- Recarga por tarjeta
- Retirada a cuenta bancaria
- Transferencia interna


C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [14]:
# La consulta SQL
query_datos_completo = """
WITH usuarios_diarios AS (
    -- Usuarios únicos por día (al menos una compra al día)
    SELECT 
        f.Id_fecha,
        COUNT(DISTINCT 
            CASE WHEN 
                (o.Operacion IN ('Pago a usuario', 'Cobro desde QR') AND 
                 (
                   (du_origen.Tipo_usuario = 'usuario' AND 
                    du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
                   OR
                   (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND 
                    du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
                 ))
                THEN f.Usuario_emisor
            END
        ) as usuarios_unicos_dia
    FROM fact_table f
    JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    GROUP BY f.Id_fecha
),
usuarios_semanales AS (
    -- Usuarios únicos por día de la semana (promedio de usuarios únicos diarios)
    SELECT 
        DAYOFWEEK(STR_TO_DATE(Id_fecha, '%Y%m%d')) as dia_semana,
        AVG(usuarios_unicos_dia) as promedio_usuarios_dia
    FROM usuarios_diarios
    GROUP BY DAYOFWEEK(STR_TO_DATE(Id_fecha, '%Y%m%d'))
),
usuarios_mensuales AS (
    -- Usuarios únicos por mes (al menos una compra en el mes)
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        SUBSTRING(f.Id_fecha, 5, 2) as mes,
        COUNT(DISTINCT 
            CASE WHEN 
                (o.Operacion IN ('Pago a usuario', 'Cobro desde QR') AND 
                 (
                   (du_origen.Tipo_usuario = 'usuario' AND 
                    du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
                   OR
                   (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND 
                    du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
                 ))
                THEN f.Usuario_emisor
            END
        ) as usuarios_unicos_mes
    FROM fact_table f
    JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE SUBSTRING(f.Id_fecha, 1, 4) = '2024'
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6), SUBSTRING(f.Id_fecha, 5, 2)
)
SELECT 
    ud.Id_fecha,
    CASE us.dia_semana
        WHEN 1 THEN 'Domingo'
        WHEN 2 THEN 'Lunes'
        WHEN 3 THEN 'Martes'
        WHEN 4 THEN 'Miércoles'
        WHEN 5 THEN 'Jueves'
        WHEN 6 THEN 'Viernes'
        WHEN 7 THEN 'Sábado'
    END as dia_semana,
    CASE um.mes
        WHEN '01' THEN 'Enero'
        WHEN '02' THEN 'Febrero'
        WHEN '03' THEN 'Marzo'
        WHEN '04' THEN 'Abril'
        WHEN '05' THEN 'Mayo'
        WHEN '06' THEN 'Junio'
        WHEN '07' THEN 'Julio'
        WHEN '08' THEN 'Agosto'
        WHEN '09' THEN 'Septiembre'
        WHEN '10' THEN 'Octubre'
        WHEN '11' THEN 'Noviembre'
        WHEN '12' THEN 'Diciembre'
    END as mes,
    ud.usuarios_unicos_dia,
    us.promedio_usuarios_dia,
    um.usuarios_unicos_mes
FROM usuarios_diarios ud
JOIN usuarios_semanales us ON DAYOFWEEK(STR_TO_DATE(ud.Id_fecha, '%Y%m%d')) = us.dia_semana
JOIN usuarios_mensuales um ON SUBSTRING(ud.Id_fecha, 5, 2) = um.mes
ORDER BY ud.Id_fecha;
"""

# Ejecutar la consulta
df = ejecutar_query(query_datos_completo)

# Convertir Id_fecha a datetime
df['fecha'] = pd.to_datetime(df['Id_fecha'], format='%Y%m%d')

# Calcular medias móviles para la evolución diaria
df['media_movil_14d'] = df['usuarios_unicos_dia'].rolling(window=14, center=True).mean()
df['media_movil_28d'] = df['usuarios_unicos_dia'].rolling(window=28, center=True).mean()

# Crear las tres gráficas
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Evolución Diaria de Usuarios Únicos en Compras (al menos una compra por día)',
        'Promedio de Usuarios Únicos por Día de la Semana',
        'Usuarios Únicos por Mes (al menos una compra en el mes)'
    ),
    vertical_spacing=0.12,
    row_heights=[0.5, 0.25, 0.25]
)

# 1. Gráfica de evolución diaria con medias móviles
fig.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['usuarios_unicos_dia'],
        name='Usuarios únicos diarios',
        line=dict(color='lightgray', width=1),
        mode='lines'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_14d'],
        name='Media móvil 14 días',
        line=dict(color='blue', width=2),
        mode='lines'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_28d'],
        name='Media móvil 28 días',
        line=dict(color='red', width=2),
        mode='lines'
    ),
    row=1, col=1
)

# 2. Gráfica de patrón semanal
orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
promedios_semanales = df.groupby('dia_semana')['promedio_usuarios_dia'].first().reindex(orden_dias)

fig.add_trace(
    go.Bar(
        x=orden_dias,
        y=promedios_semanales,
        name='Promedio por día',
        marker_color='lightblue'
    ),
    row=2, col=1
)

# 3. Gráfica de usuarios únicos mensuales
orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
usuarios_mensuales = df.groupby('mes')['usuarios_unicos_mes'].first().reindex(orden_meses)

fig.add_trace(
    go.Bar(
        x=orden_meses,
        y=usuarios_mensuales,
        name='Usuarios únicos por mes',
        marker_color='lightgreen'
    ),
    row=3, col=1
)

# Actualizar diseño
fig.update_layout(
    height=1200,
    title_text='Análisis de Usuarios Únicos en Operaciones de Compra - 2024',
    showlegend=True
)

# Actualizar ejes
fig.update_xaxes(title_text="Fecha", row=1, col=1)
fig.update_xaxes(title_text="Día de la Semana", row=2, col=1)
fig.update_xaxes(title_text="Mes", row=3, col=1)

fig.update_yaxes(title_text="Número de Usuarios Únicos por Día", row=1, col=1)
fig.update_yaxes(title_text="Promedio de Usuarios Únicos por Día", row=2, col=1)
fig.update_yaxes(title_text="Total Usuarios Únicos por Mes", row=3, col=1)

# Mostrar la figura
fig.show()

# Después de ejecutar la query y tener el DataFrame df

# Crear diccionario con toda la información
resultados = {
    "estadisticas_diarias": {
        "media_usuarios_dia": float(df['usuarios_unicos_dia'].mean()),
        "mediana_usuarios_dia": float(df['usuarios_unicos_dia'].median()),
        "maximo_usuarios_dia": float(df['usuarios_unicos_dia'].max()),
        "minimo_usuarios_dia": float(df['usuarios_unicos_dia'].min())
    },
    "datos_diarios": df[['Id_fecha', 'usuarios_unicos_dia', 'media_movil_14d', 'media_movil_28d']].to_dict('records'),
    "promedios_semanales": promedios_semanales.to_dict(),
    "usuarios_mensuales": usuarios_mensuales.to_dict()
}

# Imprimir en formato texto plano
print("\nESTADÍSTICAS DIARIAS:")
print(f"Media de usuarios únicos por día: {resultados['estadisticas_diarias']['media_usuarios_dia']:.2f}")
print(f"Mediana de usuarios únicos por día: {resultados['estadisticas_diarias']['mediana_usuarios_dia']:.2f}")
print(f"Máximo de usuarios únicos en un día: {resultados['estadisticas_diarias']['maximo_usuarios_dia']:.2f}")
print(f"Mínimo de usuarios únicos en un día: {resultados['estadisticas_diarias']['minimo_usuarios_dia']:.2f}")

print("\nDATOS DIARIOS:")
for dato in resultados['datos_diarios']:
    print(f"Fecha: {dato['Id_fecha']}, "
          f"Usuarios únicos: {dato['usuarios_unicos_dia']:.2f}, "
          f"Media móvil 14d: {dato['media_movil_14d']:.2f}, "
          f"Media móvil 28d: {dato['media_movil_28d']:.2f}")

print("\nPROMEDIOS POR DÍA DE LA SEMANA:")
for dia, promedio in resultados['promedios_semanales'].items():
    print(f"{dia}: {promedio:.2f}")

print("\nUSUARIOS ÚNICOS POR MES:")
for mes, usuarios in resultados['usuarios_mensuales'].items():
    print(f"{mes}: {usuarios:.2f}")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\mikel\AppData\Roaming\Python\Python39\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result




ESTADÍSTICAS DIARIAS:
Media de usuarios únicos por día: 194.61
Mediana de usuarios únicos por día: 222.50
Máximo de usuarios únicos en un día: 338.00
Mínimo de usuarios únicos en un día: 8.00

DATOS DIARIOS:
Fecha: 20240101, Usuarios únicos: 14.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240102, Usuarios únicos: 238.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240103, Usuarios únicos: 230.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240104, Usuarios únicos: 282.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240105, Usuarios únicos: 288.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240106, Usuarios únicos: 72.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240107, Usuarios únicos: 55.00, Media móvil 14d: nan, Media móvil 28d: nan
Fecha: 20240108, Usuarios únicos: 260.00, Media móvil 14d: 196.21, Media móvil 28d: nan
Fecha: 20240109, Usuarios únicos: 275.00, Media móvil 14d: 212.50, Media móvil 28d: nan
Fecha: 20240110, Usuari

CORRECION QUERIES LANDING

In [15]:
# 1. Query de usuarios (analyze_users)
query_users = """
WITH CategorizedUsers AS (
    SELECT 
        Id_usuario, 
        Id_fecha_alta, 
        CASE 
            WHEN Tipo_usuario IN ('autonomo', 'Empresa') THEN 'Empresas'
            ELSE Tipo_usuario
        END AS Categoria_Agrupada
    FROM dim_usuarios
    WHERE Tipo_usuario NOT IN ('Asociacion', 'ekhilur')
),
Totals AS (
    SELECT 
        Categoria_Agrupada,
        SUM(CASE WHEN Id_fecha_alta <= '20241130' THEN 1 ELSE 0 END) AS Total_Noviembre_2024,
        SUM(CASE WHEN Id_fecha_alta <= '20241231' THEN 1 ELSE 0 END) AS Total_Diciembre_2024
    FROM CategorizedUsers
    GROUP BY Categoria_Agrupada
)
SELECT 
    Categoria_Agrupada AS Categoria,
    Total_Noviembre_2024,
    Total_Diciembre_2024,
    (Total_Diciembre_2024 - Total_Noviembre_2024) AS Incremento_Absoluto,
    CASE 
        WHEN Total_Noviembre_2024 > 0 
        THEN ROUND(((Total_Diciembre_2024 - Total_Noviembre_2024) / Total_Noviembre_2024) * 100, 1)
        ELSE 0
    END AS Incremento_Porcentual
FROM Totals;
"""

# 2. Query de gasto medio mensual (analyze_monthly_average_simple)
query_monthly_avg = """
WITH UsuariosFiltrados AS (
    SELECT Id_usuario 
    FROM dim_usuarios 
    WHERE Tipo_usuario = 'usuario'
),
Fechas2024 AS (
    SELECT Id_fecha, Mes 
    FROM dim_fecha 
    WHERE Ano = 2024
),
Operaciones2024 AS (
    SELECT 
        f.Usuario_emisor, 
        f.Id_fecha, 
        f.Cantidad, 
        d.Mes,
        o.Operacion,
        du_origen.Tipo_usuario as tipo_origen,
        du_destino.Tipo_usuario as tipo_destino
    FROM fact_table f
    INNER JOIN Fechas2024 d ON f.Id_fecha = d.Id_fecha
    INNER JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    INNER JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    INNER JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    WHERE (
        -- Pago normal: usuario particular a profesional
        (o.Operacion = 'Pago a usuario' 
         AND du_origen.Tipo_usuario = 'usuario' 
         AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        OR 
        -- Pago normal entre profesionales
        (o.Operacion = 'Pago a usuario' 
         AND du_origen.Tipo_usuario IN ('Empresa', 'autonomo') 
         AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        OR
        -- Cobro desde QR
        (o.Operacion = 'Cobro desde QR' 
         AND (
             (du_origen.Tipo_usuario = 'usuario' AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
             OR 
             (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
         ))
    )
    -- Excluir explícitamente recargas y bizum entre usuarios
    AND NOT (
        o.Operacion = 'Pago a usuario' 
        AND (
            (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND du_destino.Tipo_usuario = 'usuario')  -- Recarga
            OR (du_origen.Tipo_usuario = 'usuario' AND du_destino.Tipo_usuario = 'usuario')  -- Bizum entre usuarios
        )
    )
),
GastoMensual AS (
    SELECT 
        Usuario_emisor, 
        Mes, 
        ROUND(SUM(Cantidad), 2) AS GastoMensual
    FROM Operaciones2024
    GROUP BY Usuario_emisor, Mes
)
SELECT 
    Mes,
    ROUND(AVG(GastoMensual), 2) AS Gasto_Medio_Mensual,
    COUNT(DISTINCT Usuario_emisor) as Num_Usuarios
FROM GastoMensual
GROUP BY Mes
ORDER BY Mes;
"""

# 3. Query de ahorro mensual (analyze_monthly_savings)
query_savings = """
WITH Fechas2024 AS (
    SELECT Id_fecha, Mes 
    FROM dim_fecha 
    WHERE Ano = 2024
),
Bonificaciones AS (
    SELECT 
        f.Usuario_receptor,
        f.Id_fecha,
        f.Cantidad,
        d.Mes,
        o.Operacion
    FROM fact_table f
    INNER JOIN Fechas2024 d ON f.Id_fecha = d.Id_fecha
    INNER JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE o.Operacion IN ('Bonificación por compra', 'Descuento automático')
),
AhorroMensual AS (
    SELECT 
        Usuario_receptor,
        Mes,
        ROUND(SUM(Cantidad), 2) AS AhorroMensual
    FROM Bonificaciones
    GROUP BY Usuario_receptor, Mes
)
SELECT 
    Mes,
    ROUND(AVG(AhorroMensual), 2) AS Ahorro_Medio_Mensual,
    COUNT(DISTINCT Usuario_receptor) as Num_Usuarios,
    ROUND(SUM(AhorroMensual), 2) as Total_Bonificaciones
FROM AhorroMensual
GROUP BY Mes
ORDER BY Mes;
"""

# 4. Query de totales simples (analyze_total_simple)
query_total_simple = """
WITH UltimoMes AS (
    SELECT Id_fecha 
    FROM dim_fecha 
    WHERE Ano = 2024 AND Mes = 12
),
Operaciones AS (
    SELECT 
        f.Id_transaccion,
        f.Cantidad,
        o.Operacion,
        du_origen.Tipo_usuario as tipo_origen,
        du_destino.Tipo_usuario as tipo_destino
    FROM fact_table f
    INNER JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    INNER JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    INNER JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    WHERE f.Id_fecha IN (SELECT Id_fecha FROM UltimoMes)
    AND (
        -- Caso 1: Pago normal (usuario a profesional)
        (o.Operacion = 'Pago a usuario' 
         AND du_origen.Tipo_usuario = 'usuario' 
         AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        OR
        -- Caso 2: Pago entre profesionales
        (o.Operacion = 'Pago a usuario' 
         AND du_origen.Tipo_usuario IN ('Empresa', 'autonomo') 
         AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        OR
        -- Caso 3: Cobro desde QR (puede ser usuario a profesional o entre profesionales)
        (o.Operacion = 'Cobro desde QR' 
         AND (
             (du_origen.Tipo_usuario = 'usuario' AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
             OR 
             (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
         ))
    )
)
SELECT 
    COUNT(DISTINCT Id_transaccion) AS `Número total de operaciones`, 
    ROUND(SUM(Cantidad), 2) AS `Importe total (€)`
FROM Operaciones;
"""

# 5. Query de flujo de caja (analyze_cash_flow)
query_cash_flow = """
WITH Fechas2024 AS (
    SELECT Id_fecha, Mes 
    FROM dim_fecha 
    WHERE Ano = 2024
),
Operaciones2024 AS (
    SELECT 
        f.Id_fecha, 
        f.Cantidad, 
        d.Mes,
        o.Operacion
    FROM fact_table f
    INNER JOIN Fechas2024 d ON f.Id_fecha = d.Id_fecha
    INNER JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE o.Operacion IN (
        'Recarga por tarjeta',
        'Bonificación por compra',
        'Conversión a €',
        'Cuota mensual de socio',
        'Cuota variable'
    )
),
Entradas AS (
    SELECT Mes, ROUND(SUM(Cantidad), 2) AS Entradas
    FROM Operaciones2024
    WHERE Operacion IN ('Recarga por tarjeta', 'Bonificación por compra')
    GROUP BY Mes
),
Salidas AS (
    SELECT Mes, ROUND(SUM(Cantidad), 2) AS Salidas
    FROM Operaciones2024
    WHERE Operacion IN ('Conversión a €', 'Cuota mensual de socio', 'Cuota variable')
    GROUP BY Mes
)
SELECT 
    CAST(COALESCE(e.Mes, s.Mes) AS UNSIGNED) AS Mes,
    COALESCE(e.Entradas, 0) AS Entradas,
    COALESCE(s.Salidas, 0) AS Salidas,
    COALESCE(e.Entradas, 0) - COALESCE(s.Salidas, 0) AS Balance
FROM Entradas e
LEFT JOIN Salidas s ON e.Mes = s.Mes
UNION
SELECT 
    CAST(COALESCE(e.Mes, s.Mes) AS UNSIGNED) AS Mes,
    COALESCE(e.Entradas, 0) AS Entradas,
    COALESCE(s.Salidas, 0) AS Salidas,
    COALESCE(e.Entradas, 0) - COALESCE(s.Salidas, 0) AS Balance
FROM Salidas s
LEFT JOIN Entradas e ON s.Mes = e.Mes
ORDER BY Mes;
"""

# Ejecutar queries
df_users = ejecutar_query(query_users)
df_monthly_avg = ejecutar_query(query_monthly_avg)
df_savings = ejecutar_query(query_savings)
df_total_simple = ejecutar_query(query_total_simple)
df_cash_flow = ejecutar_query(query_cash_flow)

# Crear visualizaciones del dashboard principal
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Usuarios por Categoría',
        'Gasto y Ahorro Medio Mensual',
        'Flujo de Caja Mensual',
        'Balance Mensual'
    )
)

# 1. Gráfico de usuarios
fig.add_trace(
    go.Bar(
        name='Noviembre 2024',
        x=df_users['Categoria'],
        y=df_users['Total_Noviembre_2024'],
        marker_color='#636EFA'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        name='Diciembre 2024',
        x=df_users['Categoria'],
        y=df_users['Total_Diciembre_2024'],
        marker_color='#EF553B'
    ),
    row=1, col=1
)

# 2. Gráfico de gasto y ahorro
fig.add_trace(
    go.Scatter(
        name='Gasto Medio',
        x=df_monthly_avg['Mes'],
        y=df_monthly_avg['Gasto_Medio_Mensual'],
        line=dict(color='#EF553B')
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        name='Ahorro Medio',
        x=df_savings['Mes'],
        y=df_savings['Ahorro_Medio_Mensual'],
        line=dict(color='#00CC96')
    ),
    row=1, col=2
)

# 3. Gráfico de flujo de caja
fig.add_trace(
    go.Bar(
        name='Entradas',
        x=df_cash_flow['Mes'],
        y=df_cash_flow['Entradas'],
        marker_color='#00CC96'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Bar(
        name='Salidas',
        x=df_cash_flow['Mes'],
        y=df_cash_flow['Salidas'],
        marker_color='#EF553B'
    ),
    row=2, col=1
)

# 4. Gráfico de balance
fig.add_trace(
    go.Scatter(
        name='Balance',
        x=df_cash_flow['Mes'],
        y=df_cash_flow['Balance'],
        line=dict(color='#AB63FA')
    ),
    row=2, col=2
)

# Actualizar layout del dashboard principal
fig.update_layout(
    height=800,
    showlegend=True,
    title_text="Dashboard Ekhilur 2024",
    template="plotly_white"
)

# Mostrar dashboard principal
fig.show()

# Crear figura separada para totales
fig_totals = go.Figure()

fig_totals.add_trace(go.Indicator(
    mode="number",
    value=df_total_simple.iloc[0]['Número total de operaciones'],
    title={"text": "Número total de operaciones<br>Diciembre 2024"},
    domain={'row': 0, 'column': 0}
))

fig_totals.add_trace(go.Indicator(
    mode="number",
    value=df_total_simple.iloc[0]['Importe total (€)'],
    title={"text": "Importe total<br>Diciembre 2024"},
    number={'prefix': "€"},
    domain={'row': 0, 'column': 1}
))

fig_totals.update_layout(
    grid={'rows': 1, 'columns': 2, 'pattern': "independent"},
    title_text="Totales del Último Mes",
    height=400
)

# Mostrar figura de totales
fig_totals.show()

# Imprimir estadísticas
print("\nESTADÍSTICAS RESUMEN")
print("=" * 80)

print("\nUsuarios por Categoría:")
print(df_users)

print("\nGasto Medio Mensual:")
print(df_monthly_avg)

print("\nAhorro Medio Mensual:")
print(df_savings)

print("\nTotales Último Mes:")
print(df_total_simple)

print("\nFlujo de Caja Mensual:")
print(df_cash_flow)

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.




ESTADÍSTICAS RESUMEN

Usuarios por Categoría:
  Categoria  Total_Noviembre_2024  Total_Diciembre_2024  Incremento_Absoluto  \
0   usuario                1563.0                1572.0                  9.0   
1  Empresas                 128.0                 128.0                  0.0   

   Incremento_Porcentual  
0                    0.6  
1                    0.0  

Gasto Medio Mensual:
     Mes  Gasto_Medio_Mensual  Num_Usuarios
0    1.0               244.35           860
1    2.0               224.07           849
2    3.0               240.70           849
3    4.0               236.45           836
4    5.0               250.24           884
5    6.0               238.60           854
6    7.0               234.49           852
7    8.0               190.41           773
8    9.0               226.15           868
9   10.0               255.98           914
10  11.0               237.45           922
11  12.0               296.48           949

Ahorro Medio Mensual:
     Mes  Ahor

In [18]:
# Consulta SQL para obtener datos de operaciones
query_operaciones = """
WITH operaciones_diarias AS (
    -- Operaciones e importes por día
    SELECT 
        f.Id_fecha,
        COUNT(*) as num_operaciones_dia,
        SUM(f.Cantidad) as importe_total_dia,
        AVG(f.Cantidad) as ticket_medio_dia
    FROM fact_table f
    JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE 
        SUBSTRING(f.Id_fecha, 1, 4) = '2024'
        AND (o.Operacion IN ('Pago a usuario', 'Cobro desde QR'))
        AND (
            (du_origen.Tipo_usuario = 'usuario' AND 
             du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
            OR
            (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND 
             du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        )
    GROUP BY f.Id_fecha
),
operaciones_semanales AS (
    -- Promedio de operaciones e importes por día de la semana
    SELECT 
        DAYOFWEEK(STR_TO_DATE(Id_fecha, '%Y%m%d')) as dia_semana,
        AVG(num_operaciones_dia) as promedio_operaciones_dia,
        AVG(importe_total_dia) as promedio_importe_dia,
        AVG(ticket_medio_dia) as promedio_ticket_dia
    FROM operaciones_diarias
    GROUP BY DAYOFWEEK(STR_TO_DATE(Id_fecha, '%Y%m%d'))
),
operaciones_mensuales AS (
    -- Totales mensuales
    SELECT 
        SUBSTRING(f.Id_fecha, 1, 6) as año_mes,
        SUBSTRING(f.Id_fecha, 5, 2) as mes,
        COUNT(*) as num_operaciones_mes,
        SUM(f.Cantidad) as importe_total_mes,
        AVG(f.Cantidad) as ticket_medio_mes
    FROM fact_table f
    JOIN dim_usuarios du_origen ON f.Usuario_emisor = du_origen.Id_usuario
    JOIN dim_usuarios du_destino ON f.Usuario_receptor = du_destino.Id_usuario
    JOIN dim_operaciones o ON f.Id_tipo_operacion = o.Id_tipo_operacion
    WHERE 
        SUBSTRING(f.Id_fecha, 1, 4) = '2024'
        AND (o.Operacion IN ('Pago a usuario', 'Cobro desde QR'))
        AND (
            (du_origen.Tipo_usuario = 'usuario' AND 
             du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
            OR
            (du_origen.Tipo_usuario IN ('Empresa', 'autonomo') AND 
             du_destino.Tipo_usuario IN ('Empresa', 'autonomo'))
        )
    GROUP BY SUBSTRING(f.Id_fecha, 1, 6), SUBSTRING(f.Id_fecha, 5, 2)
)
SELECT 
    od.Id_fecha,
    CASE os.dia_semana
        WHEN 1 THEN 'Domingo'
        WHEN 2 THEN 'Lunes'
        WHEN 3 THEN 'Martes'
        WHEN 4 THEN 'Miércoles'
        WHEN 5 THEN 'Jueves'
        WHEN 6 THEN 'Viernes'
        WHEN 7 THEN 'Sábado'
    END as dia_semana,
    CASE om.mes
        WHEN '01' THEN 'Enero'
        WHEN '02' THEN 'Febrero'
        WHEN '03' THEN 'Marzo'
        WHEN '04' THEN 'Abril'
        WHEN '05' THEN 'Mayo'
        WHEN '06' THEN 'Junio'
        WHEN '07' THEN 'Julio'
        WHEN '08' THEN 'Agosto'
        WHEN '09' THEN 'Septiembre'
        WHEN '10' THEN 'Octubre'
        WHEN '11' THEN 'Noviembre'
        WHEN '12' THEN 'Diciembre'
    END as mes,
    od.num_operaciones_dia,
    od.importe_total_dia,
    od.ticket_medio_dia,
    os.promedio_operaciones_dia,
    os.promedio_importe_dia,
    os.promedio_ticket_dia,
    om.num_operaciones_mes,
    om.importe_total_mes,
    om.ticket_medio_mes
FROM operaciones_diarias od
JOIN operaciones_semanales os ON DAYOFWEEK(STR_TO_DATE(od.Id_fecha, '%Y%m%d')) = os.dia_semana
JOIN operaciones_mensuales om ON SUBSTRING(od.Id_fecha, 5, 2) = om.mes
ORDER BY od.Id_fecha;
"""

# Ejecutar la consulta
df = ejecutar_query(query_operaciones)

# Convertir Id_fecha a datetime
df['fecha'] = pd.to_datetime(df['Id_fecha'], format='%Y%m%d')

# Calcular medias móviles
df['media_movil_operaciones_14d'] = df['num_operaciones_dia'].rolling(window=14, center=True).mean()
df['media_movil_operaciones_28d'] = df['num_operaciones_dia'].rolling(window=28, center=True).mean()
df['media_movil_importe_14d'] = df['importe_total_dia'].rolling(window=14, center=True).mean()
df['media_movil_importe_28d'] = df['importe_total_dia'].rolling(window=28, center=True).mean()

# Después de obtener el DataFrame, vamos a crear tres gráficas separadas para mejor visualización

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Gráfica de evolución diaria
fig1 = go.Figure()

# Añadir operaciones diarias
fig1.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['num_operaciones_dia'],
        name='Operaciones diarias',
        line=dict(color='lightgray', width=1),
        mode='lines'
    )
)

# Añadir medias móviles
fig1.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_operaciones_14d'],
        name='Media móvil 14 días',
        line=dict(color='blue', width=2),
        mode='lines'
    )
)

fig1.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_operaciones_28d'],
        name='Media móvil 28 días',
        line=dict(color='red', width=2),
        mode='lines'
    )
)

# Configurar diseño
fig1.update_layout(
    title='Evolución Diaria del Número de Operaciones - 2024',
    xaxis_title='Fecha',
    yaxis_title='Número de Operaciones',
    height=600,
    showlegend=True
)

# Mostrar primera gráfica
fig1.show()

# 2. Gráfica de importe diario
fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['importe_total_dia'],
        name='Importe diario',
        line=dict(color='lightgray', width=1),
        mode='lines'
    )
)

fig2.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_importe_14d'],
        name='Media móvil 14 días',
        line=dict(color='orange', width=2),
        mode='lines'
    )
)

fig2.add_trace(
    go.Scatter(
        x=df['fecha'],
        y=df['media_movil_importe_28d'],
        name='Media móvil 28 días',
        line=dict(color='green', width=2),
        mode='lines'
    )
)

fig2.update_layout(
    title='Evolución Diaria del Importe Total - 2024',
    xaxis_title='Fecha',
    yaxis_title='Importe Total (€)',
    height=600,
    showlegend=True
)

# Mostrar segunda gráfica
fig2.show()

# 3. Gráfica de patrón semanal
fig3 = make_subplots(specs=[[{"secondary_y": True}]])

orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
promedios_semanales = df.groupby('dia_semana')['promedio_operaciones_dia'].first().reindex(orden_dias)
promedios_importe = df.groupby('dia_semana')['promedio_importe_dia'].first().reindex(orden_dias)

fig3.add_trace(
    go.Bar(
        x=orden_dias,
        y=promedios_semanales,
        name='Promedio operaciones',
        marker_color='lightblue'
    ),
    secondary_y=False
)

fig3.add_trace(
    go.Scatter(
        x=orden_dias,
        y=promedios_importe,
        name='Promedio importe',
        line=dict(color='red', width=2),
        mode='lines+markers'
    ),
    secondary_y=True
)

fig3.update_layout(
    title='Patrón Semanal de Operaciones e Importes',
    height=600,
    showlegend=True
)

fig3.update_yaxes(title_text="Número de Operaciones", secondary_y=False)
fig3.update_yaxes(title_text="Importe Promedio (€)", secondary_y=True)

# Mostrar tercera gráfica
fig3.show()

# 4. Gráfica mensual
fig4 = make_subplots(specs=[[{"secondary_y": True}]])

orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
               'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
operaciones_mensuales = df.groupby('mes')['num_operaciones_mes'].first().reindex(orden_meses)
importes_mensuales = df.groupby('mes')['importe_total_mes'].first().reindex(orden_meses)

fig4.add_trace(
    go.Bar(
        x=orden_meses,
        y=operaciones_mensuales,
        name='Operaciones mensuales',
        marker_color='lightgreen'
    ),
    secondary_y=False
)

fig4.add_trace(
    go.Scatter(
        x=orden_meses,
        y=importes_mensuales,
        name='Importe total mensual',
        line=dict(color='purple', width=2),
        mode='lines+markers'
    ),
    secondary_y=True
)

fig4.update_layout(
    title='Evolución Mensual de Operaciones e Importes',
    height=600,
    showlegend=True
)

fig4.update_yaxes(title_text="Número de Operaciones", secondary_y=False)
fig4.update_yaxes(title_text="Importe Total Mensual (€)", secondary_y=True)

# Mostrar cuarta gráfica
fig4.show()

# Mostrar estadísticas
print("\nESTADÍSTICAS DE OPERACIONES:")
print(f"Media diaria de operaciones: {df['num_operaciones_dia'].mean():.2f}")
print(f"Mediana diaria de operaciones: {df['num_operaciones_dia'].median():.2f}")
print(f"Máximo de operaciones en un día: {df['num_operaciones_dia'].max():.0f}")
print(f"Mínimo de operaciones en un día: {df['num_operaciones_dia'].min():.0f}")

print("\nESTADÍSTICAS DE IMPORTES:")
print(f"Importe medio diario: {df['importe_total_dia'].mean():.2f}€")
print(f"Importe total del periodo: {df['importe_total_dia'].sum():.2f}€")
print(f"Ticket medio global: {df['ticket_medio_dia'].mean():.2f}€")

print("\nDÍAS CON MAYOR ACTIVIDAD:")
max_ops_day = df.loc[df['num_operaciones_dia'].idxmax()]
max_amount_day = df.loc[df['importe_total_dia'].idxmax()]
print(f"Día con más operaciones: {max_ops_day['fecha'].strftime('%d/%m/%Y')} ({max_ops_day['num_operaciones_dia']:.0f} operaciones)")
print(f"Día con mayor importe: {max_amount_day['fecha'].strftime('%d/%m/%Y')} ({max_amount_day['importe_total_dia']:.2f}€)")

C:\Users\mikel\AppData\Local\Temp\ipykernel_12748\1450764005.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\mikel\AppData\Roaming\Python\Python39\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result




ESTADÍSTICAS DE OPERACIONES:
Media diaria de operaciones: 288.73
Mediana diaria de operaciones: 330.00
Máximo de operaciones en un día: 510
Mínimo de operaciones en un día: 8

ESTADÍSTICAS DE IMPORTES:
Importe medio diario: 6843.97€
Importe total del periodo: 2504894.68€
Ticket medio global: 22.45€

DÍAS CON MAYOR ACTIVIDAD:
Día con más operaciones: 18/12/2024 (510 operaciones)
Día con mayor importe: 15/11/2024 (17496.73€)
